La librería que vamos a usar para la automatización del monitoreo de los datos, es Watchdog. Estuvimos averiguando y es una muy buena herramienta para identificar cambios en el sistema de archivos en tiempo real. Es particularmente útil cuando necesitas realizar acciones automáticas en respuesta a eventos como la creación, modificación, o eliminación de archivos y directorios. Es por eso, que nos decidimos por Watchdog

Para interactual con la base de datos, vamos a usar pyodbc. Ver con cual resulta mas fácil

In [ ]:
# Conectarse a la base de datos en cuestion

import pyodbc
import pandas as pd
from sqlalchemy import create_engine

# Configuración de la cadena de conexión
server = 'your_server_name.database.windows.net'
database = 'your_db_name'
username = 'your_username'
password = 'your_password'
driver = '{ODBC Driver 17 for SQL Server}'

# Conexión con pyodbc
conn = pyodbc.connect(f'DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password}')

# O, si prefieres usar SQLAlchemy para manejar la conexión y ejecutar el DataFrame
engine = create_engine(f'mssql+pyodbc://{username}:{password}@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server')


In [ ]:
# Leer el último registro procesado

last_ingestion = pd.read_sql('SELECT TOP 1 * FROM ingestion_control ORDER BY last_ingestion_id DESC', conn)
last_id = last_ingestion['last_ingestion_id'].iloc[0] if not last_ingestion.empty else 0


In [ ]:
# Cargar el CSV en un DataFrame de pandas
df = pd.read_csv('path/to/your/data.csv')

In [ ]:

# Filtrar los nuevos registros que no han sido procesados
new_data = df[df['id_column'] > last_id]

In [ ]:
# Insertar los nuevos datos en la base de datos
if not new_data.empty:
    new_data.to_sql('your_table_name', engine, if_exists='append', index=False)

    # Actualizar la tabla de control con el último ID procesado
    last_processed_id = new_data['id_column'].max()
    engine.execute(f"INSERT INTO ingestion_control (last_ingestion_id, last_ingestion_timestamp) VALUES ({last_processed_id}, CURRENT_TIMESTAMP)")
else:
    print("No hay nuevos datos para insertar.")